In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import json

# Replace with the path to your JSON file
file_path = 'query.json'

# Load the JSON data
with open(file_path) as file:
    data = json.load(file)

# Extract quarterly data
quarterly_data = data['quarterlyReports']

# Create DataFrame
df = pd.DataFrame(quarterly_data)

# Reset the index so that 'fiscalDateEnding' becomes a column instead of the index
df = df.reset_index(drop=True)

# If you want 'fiscalDateEnding' to be the first column, you can set it like this:
cols = ['fiscalDateEnding'] + [col for col in df if col != 'fiscalDateEnding']
df = df[cols]


In [3]:
# Now, let's create a mapping function that converts a date to its corresponding quarter
def map_date_to_quarter(date):
    year, month, _ = map(int, date.split('-'))
    if 1 <= month <= 3:
        quarter = 'Q1'
    elif 4 <= month <= 6:
        quarter = 'Q2'
    elif 7 <= month <= 9:
        quarter = 'Q3'
    else:
        quarter = 'Q4'
    return f"{year}-{quarter}"

In [4]:
# Converting the fiscalDateEnding column to year and quarter format
df['fiscalDateEnding'] = df['fiscalDateEnding'].apply(map_date_to_quarter)


In [5]:
df

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2023-Q3,USD,8023000000,14752000000,6729000000,42000000,1994000000,4458000000,1685000000,6029000000,...,521000000,572000000,1863000000,159000000,412000000,1714000000,2105000000,2275000000,2847000000,1704000000
1,2023-Q2,USD,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,...,520000000,557000000,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000
2,2023-Q1,USD,7509000000,14252000000,6743000000,30000000,256000000,4853000000,1655000000,7253000000,...,527000000,547000000,1051000000,124000000,367000000,934000000,888000000,1418000000,1965000000,927000000
3,2022-Q4,USD,9632000000,16690000000,7058000000,190000000,3294000000,4766000000,1604000000,6338000000,...,570000000,567000000,3154000000,444000000,313000000,2870000000,3109000000,3467000000,4034000000,2710000000
4,2022-Q3,USD,7430000000,14107000000,6677000000,57000000,1471000000,4391000000,1611000000,5959000000,...,586000000,577000000,-4483000000,-1287000000,295000000,-3214000000,1834000000,-4188000000,-3611000000,-3196000000
5,2022-Q2,USD,8290000000,15535000000,7246000000,120000000,2377000000,4855000000,1673000000,5913000000,...,620000000,626000000,1649000000,257000000,297000000,1465000000,1755000000,1946000000,2572000000,1392000000
6,2022-Q1,USD,7335000000,14197000000,6862000000,19000000,470000000,4597000000,1679000000,6865000000,...,631000000,625000000,694000000,-39000000,311000000,662000000,1436000000,1005000000,1630000000,733000000
7,2021-Q4,USD,9501000000,16694000000,7195000000,89000000,913000000,4903000000,1625000000,5194000000,...,749000000,632000000,2739000000,406000000,303000000,2462000000,5136000000,3042000000,3674000000,2333000000
8,2021-Q3,USD,7106000000,13251000000,6145000000,49000000,1761000000,4306000000,1606000000,6410000000,...,1037000000,647000000,906000000,-224000000,291000000,1037000000,1480000000,1196000000,1843000000,1130000000
9,2021-Q2,USD,7852000000,14218000000,6366000000,103000000,2304000000,4849000000,1641000000,6700000000,...,1050000000,630000000,1426000000,101000000,281000000,810000000,1930000000,1707000000,2337000000,1325000000


In [6]:
# Drop the 'reportedCurrency' column
df = df.drop(columns=['reportedCurrency', 'investmentIncomeNet'])

# Convert all other columns to numeric, coercing errors to NaN
for col in df.columns:
    if col != 'fiscalDateEnding':  # Skip the date column
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Now your dataframe has all numeric columns except for 'fiscalDateEnding'

# Now you can calculate the Revenue Growth and Operating Margin
df['Revenue Growth'] = df['totalRevenue'].pct_change() * 100
df['Operating Margin'] = (df['operatingIncome'] / df['totalRevenue']) * 100

# Make sure to handle NaN values as needed, for instance by filling or dropping them
df.fillna(0, inplace=True)  # This will replace NaN values with 0


In [7]:
df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,...,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,Revenue Growth,Operating Margin
0,2023-Q3,8023000000,14752000000,6729000000,42000000,1994000000,4458000000,1685000000,6029000000,-412000000,...,1863000000,159000000,412000000,1714000000,2105000000,2275000000,2847000000,1704000000,0.000000,13.516811
1,2023-Q2,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,-423000000,...,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000,4.901030,12.820679
2,2023-Q1,7509000000,14252000000,6743000000,30000000,256000000,4853000000,1655000000,7253000000,-367000000,...,1051000000,124000000,367000000,934000000,888000000,1418000000,1965000000,927000000,-7.903069,1.796239
3,2022-Q4,9632000000,16690000000,7058000000,190000000,3294000000,4766000000,1604000000,6338000000,-313000000,...,3154000000,444000000,313000000,2870000000,3109000000,3467000000,4034000000,2710000000,17.106371,19.736369
4,2022-Q3,7430000000,14107000000,6677000000,57000000,1471000000,4391000000,1611000000,5959000000,-295000000,...,-4483000000,-1287000000,295000000,-3214000000,1834000000,-4188000000,-3611000000,-3196000000,-15.476333,10.427447
5,2022-Q2,8290000000,15535000000,7246000000,120000000,2377000000,4855000000,1673000000,5913000000,-297000000,...,1649000000,257000000,297000000,1465000000,1755000000,1946000000,2572000000,1392000000,10.122634,15.300933
6,2022-Q1,7335000000,14197000000,6862000000,19000000,470000000,4597000000,1679000000,6865000000,-311000000,...,694000000,-39000000,311000000,662000000,1436000000,1005000000,1630000000,733000000,-8.612810,3.310559
7,2021-Q4,9501000000,16694000000,7195000000,89000000,913000000,4903000000,1625000000,5194000000,-303000000,...,2739000000,406000000,303000000,2462000000,5136000000,3042000000,3674000000,2333000000,17.588223,5.469031
8,2021-Q3,7106000000,13251000000,6145000000,49000000,1761000000,4306000000,1606000000,6410000000,-291000000,...,906000000,-224000000,291000000,1037000000,1480000000,1196000000,1843000000,1130000000,-20.624176,13.289563
9,2021-Q2,7852000000,14218000000,6366000000,103000000,2304000000,4849000000,1641000000,6700000000,-281000000,...,1426000000,101000000,281000000,810000000,1930000000,1707000000,2337000000,1325000000,7.297562,16.204811


In [23]:
import ast 

# Read the file content
file_path = 'ibm-google-link.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Initialize a list to store the parsed data
data_list = []

# Process each line
for line in lines:
    # Split by commas, the first two items are dates, the rest is the news list
    parts = line.strip().split(',', 2)
    if len(parts) < 3:  # If there are not enough parts, skip this line
        continue

    # Extract start date, end date, and the list of articles
    start_date, end_date, articles_str = parts
    # Parse the string of articles into a list (safely with ast.literal_eval)
    try:
        articles_list = ast.literal_eval(articles_str)
        # Add each article with its date range and URL to the data list
        for article in articles_list:
            title, link = article
            data_list.append({'End Date': end_date, 'Article Title': title, 'URL': link})
    except Exception as e:
        print(f"Error parsing line: {e}")

# Create a DataFrame
news_df = pd.DataFrame(data_list)

news_df.head()  # Display the first few entries of the DataFrame


,End Date,Article Title,URL
0,2019-01-31,ibm reports 2018 fourth-quarter and full-year ...,https://www.ibm.com/investor/att/pdf/IBM-4Q18-...
1,2019-01-31,IBM Began 2019 as the Cheapest Dow Stock,https://www.investopedia.com/ibm-began-2019-as...
2,2019-01-31,Stock heat map visualization,https://www.ibm.com/docs/en/cognos-analytics/1...
3,2019-01-31,IBM unveils first standalone quantum computer,https://www.ft.com/content/f0d63c74-12c6-11e9-...
4,2019-01-31,Accounting 10 Chapter 13 Assignment--50 Points...,https://www.chegg.com/homework-help/questions-...


In [24]:
news_df

,End Date,Article Title,URL
0,2019-01-31,ibm reports 2018 fourth-quarter and full-year ...,https://www.ibm.com/investor/att/pdf/IBM-4Q18-...
1,2019-01-31,IBM Began 2019 as the Cheapest Dow Stock,https://www.investopedia.com/ibm-began-2019-as...
2,2019-01-31,Stock heat map visualization,https://www.ibm.com/docs/en/cognos-analytics/1...
3,2019-01-31,IBM unveils first standalone quantum computer,https://www.ft.com/content/f0d63c74-12c6-11e9-...
4,2019-01-31,Accounting 10 Chapter 13 Assignment--50 Points...,https://www.chegg.com/homework-help/questions-...
...,...,...,...
585,2023-11-28,"IBM,AAPL - Stock Price, Quote - CNBC","https://www.cnbc.com/quotes/IBM,AAPL"
586,2023-11-28,IBM Share Price Live Today: IBM Stock Price Li...,https://www.moneycontrol.com/us-markets/stockp...
587,2023-11-28,International Business Machines Corporation (I...,https://nz.finance.yahoo.com/quote/IBM/news?p=...
588,2023-11-28,International Business Machines - IBM - Stock ...,https://www.zacks.com/stock/quote/IBM


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from urllib.parse import urlparse, unquote
import pandas as pd

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')
model = AutoModelForSequenceClassification.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')

# Function to perform sentiment analysis
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.logits.detach().numpy()

# Function to extract the slug from URL
def get_url_slug(url):
    parsed_url = urlparse(url)
    # The following line handles cases where the URL ends with '...'
    return unquote(parsed_url.path.split('/')[-1]).replace('-', ' ').replace('...', '')

# Assuming 'news_df' is your DataFrame with the relevant data
# Apply sentiment analysis to the combined text of the title and URL slug
news_df['Combined Sentiment'] = news_df.apply(lambda row: get_sentiment(row['Article Title'] + " " + get_url_slug(row['URL'])), axis=1)

# Display the DataFrame
print(news_df[['End Date', 'Article Title', 'URL', 'Combined Sentiment']])


       End Date                                      Article Title  \
0    2019-01-31  ibm reports 2018 fourth-quarter and full-year ...   
1    2019-01-31           IBM Began 2019 as the Cheapest Dow Stock   
2    2019-01-31                       Stock heat map visualization   
3    2019-01-31      IBM unveils first standalone quantum computer   
4    2019-01-31  Accounting 10 Chapter 13 Assignment--50 Points...   
..          ...                                                ...   
585  2023-11-28               IBM,AAPL - Stock Price, Quote - CNBC   
586  2023-11-28  IBM Share Price Live Today: IBM Stock Price Li...   
587  2023-11-28  International Business Machines Corporation (I...   
588  2023-11-28  International Business Machines - IBM - Stock ...   
589  2023-11-28               Operations Consulting Services | IBM   

                                                   URL  \
0    https://www.ibm.com/investor/att/pdf/IBM-4Q18-...   
1    https://www.investopedia.com/ibm-began

In [26]:
import torch.nn.functional as F

def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = F.softmax(outputs.logits, dim=-1)
    return probabilities.detach().numpy()


In [27]:
# Assuming 'news_df' is your DataFrame
# Update the sentiment analysis to include softmax
news_df['Combined Sentiment'] = news_df.apply(lambda row: get_sentiment(row['Article Title'] + " " + get_url_slug(row['URL'])), axis=1)

# Convert the probabilities to a class label
def get_label(probabilities):
    # The classes might be in the order of [negative, neutral, positive]
    labels = ['negative', 'neutral', 'positive']
    return labels[probabilities.argmax()]

# Apply the get_label function to each row in 'Combined Sentiment'
news_df['Sentiment Label'] = news_df['Combined Sentiment'].apply(get_label)


In [28]:
# Assuming 'news_df' is your DataFrame and it has a 'Sentiment Label' column
neutral_count = news_df['Sentiment Label'].value_counts().get('neutral', 0)
print(f"Number of neutral sentiments: {neutral_count}")


Number of neutral sentiments: 470


In [29]:
news_df

,End Date,Article Title,URL,Combined Sentiment,Sentiment Label
0,2019-01-31,ibm reports 2018 fourth-quarter and full-year ...,https://www.ibm.com/investor/att/pdf/IBM-4Q18-...,"[[7.240594e-05, 0.9998808, 4.6871734e-05]]",neutral
1,2019-01-31,IBM Began 2019 as the Cheapest Dow Stock,https://www.investopedia.com/ibm-began-2019-as...,"[[5.684991e-05, 0.9998903, 5.2797248e-05]]",neutral
2,2019-01-31,Stock heat map visualization,https://www.ibm.com/docs/en/cognos-analytics/1...,"[[7.666438e-05, 0.99987495, 4.8390826e-05]]",neutral
3,2019-01-31,IBM unveils first standalone quantum computer,https://www.ft.com/content/f0d63c74-12c6-11e9-...,"[[5.821864e-05, 0.9998871, 5.4657616e-05]]",neutral
4,2019-01-31,Accounting 10 Chapter 13 Assignment--50 Points...,https://www.chegg.com/homework-help/questions-...,"[[6.608352e-05, 0.99988914, 4.4810313e-05]]",neutral
...,...,...,...,...,...
585,2023-11-28,"IBM,AAPL - Stock Price, Quote - CNBC","https://www.cnbc.com/quotes/IBM,AAPL","[[7.570761e-05, 0.99987924, 4.5029286e-05]]",neutral
586,2023-11-28,IBM Share Price Live Today: IBM Stock Price Li...,https://www.moneycontrol.com/us-markets/stockp...,"[[8.0751604e-05, 0.9998642, 5.50392e-05]]",neutral
587,2023-11-28,International Business Machines Corporation (I...,https://nz.finance.yahoo.com/quote/IBM/news?p=...,"[[8.855111e-05, 0.99985456, 5.68392e-05]]",neutral
588,2023-11-28,International Business Machines - IBM - Stock ...,https://www.zacks.com/stock/quote/IBM,"[[7.976962e-05, 0.9998714, 4.888977e-05]]",neutral


In [30]:
news_df.shape

(590, 5)

In [31]:
# Define a basic set of finance-related keywords
finance_keywords = [
    "earnings", "acquisition", "merger", "revenue", "bankruptcy", "stock", "share", 
    "investment", "economy", "market", "dividend", "portfolio", "capital", "trading", 
    "valuation", "fiscal", "equity", "bond", "commodity", "index", "hedge", "funds", 
    "bullish", "bearish", "volatility", "leverage", "derivative", "yield", "rating",
    "credit", "debt", "stake", "IPO", "offer", "growth", "risk", "return", "interest", 
    "rate", "policy", "inflation", "deflation", "GDP", "regulation", "compliance", 
    "audit", "tax", "expense", "liability", "asset", "amortization", "depreciation", 
    "ledger", "arbitrage", "forex", "exchange", "short", "long", "call", "put", "trade",
    "sector", "industry", "benchmark", "spread", "futures", "options", "maturity", 
    "duration", "default", "insolvency", "liquidity", "solvency", "profit", "loss", 
    "forecast", "projection", "divestiture", "stakeholder", "shareholder", "creditor", 
    "debtor", "dilution", "restructuring", "turnaround", "buyout", "leverage", 
    "due diligence", "equity", "debt", "merger", "acquisition", "synergy", 
    "consolidation", "spin-off", "split-off", "carve-out", "integration", "disposal", 
    "joint venture", "strategic alliance", "partnership"
]

# Search function for finance keywords in titles
def search_finance_keywords(titles, keywords):
    keyword_matches = []
    for title in titles:
        title_keywords = [keyword for keyword in keywords if keyword.lower() in title.lower()]
        keyword_matches.append(title_keywords)
    return keyword_matches

# Search the article titles for finance keywords and add the results as a new column
news_df['Finance Keywords'] = search_finance_keywords(news_df['Article Title'], finance_keywords)

news_df  # Display the updated DataFrame with finance keywords



,End Date,Article Title,URL,Combined Sentiment,Sentiment Label,Finance Keywords
0,2019-01-31,ibm reports 2018 fourth-quarter and full-year ...,https://www.ibm.com/investor/att/pdf/IBM-4Q18-...,"[[7.240594e-05, 0.9998808, 4.6871734e-05]]",neutral,[]
1,2019-01-31,IBM Began 2019 as the Cheapest Dow Stock,https://www.investopedia.com/ibm-began-2019-as...,"[[5.684991e-05, 0.9998903, 5.2797248e-05]]",neutral,[stock]
2,2019-01-31,Stock heat map visualization,https://www.ibm.com/docs/en/cognos-analytics/1...,"[[7.666438e-05, 0.99987495, 4.8390826e-05]]",neutral,[stock]
3,2019-01-31,IBM unveils first standalone quantum computer,https://www.ft.com/content/f0d63c74-12c6-11e9-...,"[[5.821864e-05, 0.9998871, 5.4657616e-05]]",neutral,[put]
4,2019-01-31,Accounting 10 Chapter 13 Assignment--50 Points...,https://www.chegg.com/homework-help/questions-...,"[[6.608352e-05, 0.99988914, 4.4810313e-05]]",neutral,[]
...,...,...,...,...,...,...
585,2023-11-28,"IBM,AAPL - Stock Price, Quote - CNBC","https://www.cnbc.com/quotes/IBM,AAPL","[[7.570761e-05, 0.99987924, 4.5029286e-05]]",neutral,[stock]
586,2023-11-28,IBM Share Price Live Today: IBM Stock Price Li...,https://www.moneycontrol.com/us-markets/stockp...,"[[8.0751604e-05, 0.9998642, 5.50392e-05]]",neutral,"[stock, share]"
587,2023-11-28,International Business Machines Corporation (I...,https://nz.finance.yahoo.com/quote/IBM/news?p=...,"[[8.855111e-05, 0.99985456, 5.68392e-05]]",neutral,[stock]
588,2023-11-28,International Business Machines - IBM - Stock ...,https://www.zacks.com/stock/quote/IBM,"[[7.976962e-05, 0.9998714, 4.888977e-05]]",neutral,[stock]


In [32]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained('dslim/bert-base-NER')
model = AutoModelForTokenClassification.from_pretrained('dslim/bert-base-NER')

# Create a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Function to get NER results
def extract_ner(articles):
    ner_results = []
    for article in articles:
        ner_info = ner_pipeline(article)
        entities = [(entity['word'], entity['entity_group']) for entity in ner_info]
        ner_results.append(entities)
    return ner_results

# Apply the function to the 'Article Title' column
news_df['NER Results'] = extract_ner(news_df['Article Title'].tolist())

# Display the DataFrame with NER results
news_df[['Article Title', 'NER Results']]


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,Article Title,NER Results
0,ibm reports 2018 fourth-quarter and full-year ...,[]
1,IBM Began 2019 as the Cheapest Dow Stock,"[(IBM Began, ORG), (Ch, MISC), (##eapest Dow S..."
2,Stock heat map visualization,[]
3,IBM unveils first standalone quantum computer,"[(IBM, ORG)]"
4,Accounting 10 Chapter 13 Assignment--50 Points...,"[(Ch, ORG), (##g, ORG)]"
...,...,...
585,"IBM,AAPL - Stock Price, Quote - CNBC","[(IBM, ORG), (AAPL, ORG), (CNBC, ORG)]"
586,IBM Share Price Live Today: IBM Stock Price Li...,"[(IBM, ORG), (IBM, ORG), (News, ORG)]"
587,International Business Machines Corporation (I...,"[(International Business Machines Corporation,..."
588,International Business Machines - IBM - Stock ...,"[(International Business Machines, ORG), (IBM,..."


In [33]:
news_df.shape

(590, 7)

In [34]:
# Apply the mapping to the 'End Date' in news_df_mock (we'll replace this with the actual news_df later)
news_df['Quarter'] = news_df['End Date'].apply(map_date_to_quarter)


In [35]:
# Define aggregation functions for different features
def aggregate_text_data(df):
    # Sentiment score aggregation (taking the positive sentiment score as the point of interest)
    df['Positive Sentiment Score'] = df['Combined Sentiment'].apply(lambda x: x[0][2])
    sentiment_score_agg = df.groupby('Quarter')['Positive Sentiment Score'].mean().reset_index(name='Average Positive Sentiment')

    # Sentiment label aggregation (assuming we want the count of each label per quarter)
    sentiment_label_agg = df.groupby(['Quarter', 'Sentiment Label']).size().unstack(fill_value=0).reset_index()

    # NER aggregation (concatenating all NER results into a list per quarter)
    df['NER Results'] = df['NER Results'].apply(lambda x: [i[0] for i in x])  # Assuming we want just the text of the NER
    ner_agg = df.groupby('Quarter')['NER Results'].agg(sum).reset_index(name='Aggregated NER Results')

    # Keywords aggregation (concatenating all Finance Keywords into a list per quarter)
    df['Finance Keywords'] = df['Finance Keywords'].apply(lambda x: [i for i in x])  # Assuming we want just the text of the keywords
    keywords_agg = df.groupby('Quarter')['Finance Keywords'].agg(sum).reset_index(name='Aggregated Finance Keywords')

    # Merge all the aggregated dataframes on the 'Quarter' column
    aggregated_data = sentiment_score_agg.merge(sentiment_label_agg, on='Quarter', how='left')
    aggregated_data = aggregated_data.merge(ner_agg, on='Quarter', how='left')
    aggregated_data = aggregated_data.merge(keywords_agg, on='Quarter', how='left')

    return aggregated_data

# Apply the aggregation function to the news_df DataFrame
# Replace news_df with the actual DataFrame when running this code
aggregated_text_features = aggregate_text_data(news_df)

# Display the aggregated features
aggregated_text_features.head()


,Quarter,Average Positive Sentiment,negative,neutral,positive,Aggregated NER Results,Aggregated Finance Keywords
0,2019-Q1,0.043641,4,25,1,"[IBM Began, Ch, ##eapest Dow Stock, IBM, Ch, #...","[stock, stock, put, stock, stock, offer, stock..."
1,2019-Q2,0.129273,1,25,4,"[IBM, IBM AI, IBM, IBM Ent Hold, IBM Stock Opt...","[stock, stock, options, stock, rate, stock, ca..."
2,2019-Q3,0.194482,0,24,6,"[IBM, Red Hat, IBM, Red Hat, IBM, IBM, IBM, In...","[acquisition, acquisition, acquisition, acquis..."
3,2019-Q4,0.199876,2,22,6,"[IBM REP, IBM, IBM, AI, IBM, ##g, Boeing, P, #...","[stock, stock, market, market, market, share, ..."
4,2020-Q1,0.231825,1,22,7,"[Warren Buffett, IBM, Apple, IBM R, J & J, Mod...","[stock, market, stock, stock, stock, stock, st..."


In [36]:
aggregated_text_features.shape

(20, 7)

In [37]:
# Assuming 'df' is your financial data DataFrame and 'aggregated_text_features' is your text data DataFrame

# Convert the 'fiscalDateEnding' to a string type to match the Quarter column in aggregated_text_features
df['fiscalDateEnding'] = df['fiscalDateEnding'].astype(str)

# Rename the 'fiscalDateEnding' column in df to 'Quarter' to match the aggregated_text_features DataFrame
df.rename(columns={'fiscalDateEnding': 'Quarter'}, inplace=True)

# Merge the dataframes on the 'Quarter' column
merged_data = pd.merge(df, aggregated_text_features, on='Quarter', how='left')

# Now merged_data contains the combined financial and text data
# You can now save or continue processing merged_data as needed


In [38]:
merged_data

,Quarter,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,...,ebitda,netIncome,Revenue Growth,Operating Margin,Average Positive Sentiment,negative,neutral,positive,Aggregated NER Results,Aggregated Finance Keywords
0,2023-Q3,8023000000,14752000000,6729000000,42000000,1994000000,4458000000,1685000000,6029000000,-412000000,...,2847000000,1704000000,0.000000,13.516811,0.278318,2.0,19.0,9.0,"[NFLX, T, ##SLA, IBM, IBM, IBM, IBM, IBM, Repo...","[stock, earnings, earnings, share, revenue, st..."
1,2023-Q2,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,-423000000,...,2982000000,1583000000,4.901030,12.820679,0.151786,2.0,23.0,5.0,"[TSLA, IBM, AXP, T, IBM, IBM, IBM, IBM, First,...","[stock, earnings, profit, earnings, stock, mar..."
2,2023-Q1,7509000000,14252000000,6743000000,30000000,256000000,4853000000,1655000000,7253000000,-367000000,...,1965000000,927000000,-7.903069,1.796239,0.099418,2.0,25.0,3.0,"[IBM, IBM, Tech, Tesla, IBM, ServiceNow, Levi,...","[earnings, earnings, stock, stock, stock, trad..."
3,2022-Q4,9632000000,16690000000,7058000000,190000000,3294000000,4766000000,1604000000,6338000000,-313000000,...,4034000000,2710000000,17.106371,19.736369,0.099966,1.0,26.0,3.0,"[International Business Machines Corporation, ...","[stock, earnings, exchange, stock, revenue, in..."
4,2022-Q3,7430000000,14107000000,6677000000,57000000,1471000000,4391000000,1611000000,5959000000,-295000000,...,-3611000000,-3196000000,-15.476333,10.427447,0.088188,3.0,24.0,3.0,"[Twitter, IBM, IBM, IBM, IBM, Finance, IBM Blo...","[stock, share, forecast, trading, earnings, st..."
5,2022-Q2,8290000000,15535000000,7246000000,120000000,2377000000,4855000000,1673000000,5913000000,-297000000,...,2572000000,1392000000,10.122634,15.300933,0.067127,2.0,26.0,2.0,"[IBM, IBM, IBM, IBM, The Register, IBM, IBM, I...","[stock, earnings, stock, share, stock, share, ..."
6,2022-Q1,7335000000,14197000000,6862000000,19000000,470000000,4597000000,1679000000,6865000000,-311000000,...,1630000000,733000000,-8.612810,3.310559,0.166318,1.0,24.0,5.0,"[IBM, IBM, IBM, IBM, IBM, ##g, ETFMG, IBM, IBM...","[investment, revenue, share, growth, risk, sha..."
7,2021-Q4,9501000000,16694000000,7195000000,89000000,913000000,4903000000,1625000000,5194000000,-303000000,...,3674000000,2333000000,17.588223,5.469031,0.119884,3.0,23.0,4.0,"[IBM, Kyndryl, 10, IBM Board of Directors, Kyn...","[stock, earnings, stock, inflation, stock, sto..."
8,2021-Q3,7106000000,13251000000,6145000000,49000000,1761000000,4306000000,1606000000,6410000000,-291000000,...,1843000000,1130000000,-20.624176,13.289563,0.196403,2.0,22.0,6.0,[International Business Machines Corporation C...,"[stock, stock, rate, stock, earnings, trade, r..."
9,2021-Q2,7852000000,14218000000,6366000000,103000000,2304000000,4849000000,1641000000,6700000000,-281000000,...,2337000000,1325000000,7.297562,16.204811,0.132772,0.0,26.0,4.0,"[UNIT, E, Qiskit Finance, Qiskit Finance, IBM,...","[exchange, stock, share, industry, stock, shar..."


In [69]:
# Load the dataset again
data = merged_data

# Calculate the correlation matrix
correlation_matrix = data.corr()

# Select upper triangle of correlation matrix
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.8, excluding the four specified columns
excluded_columns = ['Operating Margin', 'Revenue Growth', 'netIncome', 'ebitda']
to_drop = [column for column in upper.columns if any(upper[column] > 0.8) and column not in excluded_columns]

# Drop the highly correlated features excluding the specified ones
data_reduced = data.drop(columns=to_drop)

# Check for missing values in the dataset
missing_values = data_reduced.isnull().sum()

# Drop the text columns 'Aggregated NER Results' and 'Aggregated Finance Keywords'
data_reduced = data_reduced.drop(columns=['Aggregated NER Results', 'Aggregated Finance Keywords'])

# Check for categorical variables that need encoding
categorical_columns = data_reduced.select_dtypes(include=['object']).columns.tolist()

# Now let's proceed with the next steps
missing_values, categorical_columns, data_reduced.columns.tolist()


C:\Users\saa\AppData\Local\Temp\ipykernel_15820\2662409890.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data.corr()


(Quarter                            0
 grossProfit                        0
 costofGoodsAndServicesSold         0
 operatingIncome                    0
 sellingGeneralAndAdministrative    0
 researchAndDevelopment             0
 operatingExpenses                  0
 netInterestIncome                  0
 interestIncome                     0
 interestExpense                    0
 nonInterestIncome                  0
 otherNonOperatingIncome            0
 depreciation                       0
 depreciationAndAmortization        0
 incomeBeforeTax                    0
 incomeTaxExpense                   0
 comprehensiveIncomeNetOfTax        0
 ebitda                             0
 netIncome                          0
 Revenue Growth                     0
 Operating Margin                   0
 Average Positive Sentiment         1
 negative                           1
 neutral                            1
 Aggregated NER Results             1
 Aggregated Finance Keywords        1
 dtype: int6

In [70]:
# Remove rows with missing values
data_cleaned = data_reduced.dropna()

# Encode the 'Quarter' column numerically
# Sort by 'Quarter' to ensure the correct order
data_cleaned_sorted = data_cleaned.sort_values('Quarter')

# Create a mapping for quarters to numerical values
quarters = data_cleaned_sorted['Quarter'].unique()
quarter_mapping = {quarter: index for index, quarter in enumerate(quarters, 1)}

# Apply the mapping to the 'Quarter' column
data_cleaned_sorted['Quarter'] = data_cleaned_sorted['Quarter'].map(quarter_mapping)

# Confirm that the four specified columns are still in the dataset and display the first few rows
specified_columns = ['Operating Margin', 'Revenue Growth', 'netIncome', 'ebitda']
remaining_columns = data_cleaned_sorted.columns.tolist()

specified_columns_present = all(column in remaining_columns for column in specified_columns)
specified_columns_present, data_cleaned_sorted.head()


(True,
     Quarter  grossProfit  costofGoodsAndServicesSold  operatingIncome  \
 18        1   8043000000                    55000000       1616000000   
 17        2   9010000000                    74000000       1950000000   
 16        3   8336000000                    75000000       1872000000   
 15        4   6145000000                   387000000       4048000000   
 14        5   7922000000                    76000000       -145000000   
 
     sellingGeneralAndAdministrative  researchAndDevelopment  \
 18                       4691000000              1433000000   
 17                       5456000000              1407000000   
 16                       5024000000              1553000000   
 15                       3553000000              1517000000   
 14                       5955000000              1625000000   
 
     operatingExpenses  netInterestIncome  interestIncome  interestExpense  \
 18         6427000000         -210000000      70000000.0        210000000   
 17  

In [71]:
# Set up features and target for predicting 'Revenue Growth'
X = data_cleaned_sorted.drop('Revenue Growth', axis=1)  # Features
y = data_cleaned_sorted['Revenue Growth']  # Target variable

# Train the model on the data up to Q3 of 2023
X_train = X[X['Quarter'] < 20]  # Features for training
y_train = y[X['Quarter'] < 20]  # Target for training

# Initialize the Linear Regression model
model_rg = LinearRegression()

# Fit the model on the training data
model_rg.fit(X_train, y_train)

# Create a feature set for Q4 of 2023, incrementing the Quarter
X_pred_rg = X_train.tail(1).copy()
X_pred_rg['Quarter'] = 20

# Predict the 'Revenue Growth' for Q4 of 2023
predicted_revenue_growth = model_rg.predict(X_pred_rg)

predicted_revenue_growth[0]


16.954494869901737

In [72]:
# Set up features and target for predicting 'Operating Margin'
X = data_cleaned_sorted.drop('Operating Margin', axis=1)  # Features
y = data_cleaned_sorted['Operating Margin']  # Target variable

# Train the model on the data up to Q3 of 2023
X_train = X[X['Quarter'] < 20]  # Features for training
y_train = y[X['Quarter'] < 20]  # Target for training

# Initialize the Linear Regression model
model_om = LinearRegression()

# Fit the model on the training data
model_om.fit(X_train, y_train)

# Create a feature set for Q4 of 2023, incrementing the Quarter
X_pred_om = X_train.tail(1).copy()
X_pred_om['Quarter'] = 20

# Predict the 'Operating Margin' for Q4 of 2023
predicted_operating_margin = model_om.predict(X_pred_om)

predicted_operating_margin[0]


13.478603453660128

In [73]:
# Set up features and target for predicting 'Net Income'
X = data_cleaned_sorted.drop('netIncome', axis=1)  # Features
y = data_cleaned_sorted['netIncome']  # Target variable

# Train the model on the data up to Q3 of 2023
X_train = X[X['Quarter'] < 20]  # Features for training
y_train = y[X['Quarter'] < 20]  # Target for training

# Initialize the Linear Regression model
model_ni = LinearRegression()

# Fit the model on the training data
model_ni.fit(X_train, y_train)

# Create a feature set for Q4 of 2023, incrementing the Quarter
X_pred_ni = X_train.tail(1).copy()
X_pred_ni['Quarter'] = 20

# Predict the 'Net Income' for Q4 of 2023
predicted_net_income = model_ni.predict(X_pred_ni)

predicted_net_income[0]


1704000000.0000017

In [74]:
# Set up features and target for predicting 'EBITDA'
X = data_cleaned_sorted.drop('ebitda', axis=1)  # Features
y = data_cleaned_sorted['ebitda']  # Target variable

# Train the model on the data up to Q3 of 2023
X_train = X[X['Quarter'] < 20]  # Features for training
y_train = y[X['Quarter'] < 20]  # Target for training

# Initialize the Linear Regression model
model_ebitda = LinearRegression()

# Fit the model on the training data
model_ebitda.fit(X_train, y_train)

# Create a feature set for Q4 of 2023, incrementing the Quarter
X_pred_ebitda = X_train.tail(1).copy()
X_pred_ebitda['Quarter'] = 20

# Predict the 'EBITDA' for Q4 of 2023
predicted_ebitda = model_ebitda.predict(X_pred_ebitda)

predicted_ebitda[0]


2846999999.9999895